This is my first CNN network, so for some of you it can be too simple, but my aim is to explain every step we need to do to create one.

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread

import os
from PIL import Image

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

In [ ]:
des = pd.read_json('/kaggle/input/cassava-leaf-disease-classification/label_num_to_disease_map.json',
                  lines=True)
labels = des.T
labels.columns = ["labels_description"]

In [ ]:
labels

In [ ]:
img_train_path = "/kaggle/input/cassava-leaf-disease-classification/train_images"
img_test_path = "/kaggle/input/cassava-leaf-disease-classification/test_images"

In [ ]:
train = pd.read_csv("/kaggle/input/cassava-leaf-disease-classification/train.csv")

In [ ]:
# Load a image
path = "/kaggle/input/cassava-leaf-disease-classification/train_images/1235188286.jpg"

single_picture = tf.keras.preprocessing.image.load_img(path, grayscale=False, color_mode="rgb", target_size=None, interpolation="nearest")
single_picture

In [ ]:
os.listdir(img_train_path)[0]

In [ ]:
picture_path = img_train_path+"/"+"1235188286.jpg"

In [ ]:
picture_path

In [ ]:
# imread(picture_path)

In [ ]:
train[train['image_id'] == "1235188286.jpg"]

In [ ]:
plt.title(labels.iloc[2].values[0])
plt.imshow(imread(picture_path))

In [ ]:
len(os.listdir(img_train_path))

### CSV train

In [ ]:
sns.countplot(x=train['label'])

In [ ]:
# train['label'] = train['label'].astype(str)

In [ ]:
class_3_picture = img_train_path+'/'+ train[train['label']==3]['image_id'][4]

In [ ]:
picture_class_3 = imread(class_3_picture)
plt.imshow(picture_class_3)

In [ ]:
# Test picture
test_picture_path = img_test_path+"/"+ os.listdir(img_test_path)[0]

plt.imshow(imread(test_picture_path))

### Shape of our pictures

It is very important to check if the pictures have the same size as CNN won't be able to work with different sizes.

In [ ]:
# dim1 = []
# dim2 = []

#for image_filename in os.listdir(img_train_path):
    
   # img = imread(img_train_path+"/"+ image_filename)
   # d1, d2, colors = img.shape
   # dim1.append(d1)
   # dim2.append(d2)

In [ ]:
#print(f"Size of all pictures is: ({dim1[0]},{dim2[0]})")

If the pictures have different size we have to resize it to e.g. mean of those dimentions. Nice way to visualize it is by seaborn jointplot.

In [ ]:
SHAPE = 456
IMAGE_SHAPE = (456, 456, 3)
BATCH_SIZE = 15

### Manipulating images

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# help(ImageDataGenerator)

In [ ]:
# imread(picture_path).max()

In [ ]:
image_gen = ImageDataGenerator(rotation_range=40,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               rescale=1/255, # we need to normalize the data
                               shear_range=0.2,
                               zoom_range=0.2,
                               horizontal_flip=True,
                               fill_mode="nearest")

In [ ]:
plt.imshow(image_gen.random_transform(picture_class_3))

In [ ]:
dict_map = {int(i): label for i, label in enumerate(labels['labels_description'].values)}
train["class_name"] = train['label'].map(dict_map)

In [ ]:
# Split into train/test set
from sklearn.model_selection import train_test_split

train, test = train_test_split(train, test_size=0.05, random_state=45, stratify=train['class_name'])

In [ ]:
train_set = image_gen.flow_from_dataframe(  
                                         train,
                                        directory=img_train_path,
                                        seed=42,
                                        x_col='image_id',
                                        y_col='class_name',
                                        target_size = (SHAPE, SHAPE),
                                        class_mode='categorical',
                                        interpolation='nearest',
                                        shuffle = True,
                                        batch_size = BATCH_SIZE,
                                    )

In [ ]:
# Do the same for our test set
datagen_val = ImageDataGenerator(
    preprocessing_function = tf.keras.applications.efficientnet.preprocess_input,
)

test_set = datagen_val.flow_from_dataframe(
    test,
    directory=img_train_path,
    seed=42,
    x_col='image_id',
    y_col='class_name',
    target_size = (SHAPE, SHAPE),
    class_mode='categorical',
    interpolation='nearest',
    batch_size=BATCH_SIZE,    
)

### Creating CNN model

In [ ]:
early_stop = EarlyStopping(monitor='val_loss',mode='min', patience=2, )

In [ ]:
IMAGE_SHAPE

In [ ]:
def create_model():
    
    # Instantiate ann model
    model = Sequential()
    
    # Add Convolution layer first
    model.add(Conv2D(filters=64, kernel_size=(3,3), input_shape=IMAGE_SHAPE, activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2)))
    
    # Add Convolution layer first
    model.add(Conv2D(filters=128, kernel_size=(3,3), activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2)))
    
    # Add Convolution layer first
    model.add(Conv2D(filters=128, kernel_size=(3,3), activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2)))
    
    # Flatten out the model
    model.add(Flatten())
    
    # Add Dense layer now
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    
    # Add output layer
    model.add(Dense(5, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
ann_model = create_model()

In [ ]:
ann_model.summary()

In [ ]:
train_set.image_shape

In [ ]:
ann_model.fit(train_set, 
              epochs=10, 
              validation_data=test_set, 
              callbacks=[early_stop])

In [ ]:
ann_model.save("First_CNN_model.h5")

In [ ]:
# imread(img_test_path+"/"+test_images[0])

## Submission

In [ ]:
final_model = tf.keras.models.load_model("First_CNN_model.h5")

In [ ]:
test_images = os.listdir(img_test_path)

predictions = []

for image in test_images:
    img = Image.open(img_test_path+"/"+ image)
    img = img.resize((SHAPE, SHAPE))
    img = np.expand_dims(img, axis=0)
    predictions.extend(final_model.predict(img).argmax(axis = 1))
    
    
sub = pd.DataFrame({'image_id': test_images, 'label': predictions})
display(sub)
sub.to_csv('submission.csv', index = False)